In [300]:
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from tabulate import tabulate

### Input document name and add content to postings list

In [301]:
content_tokenizer = RegexpTokenizer('<TEXT.*?>(.*?)</TEXT>')
article_tokenizer = RegexpTokenizer('<REUTERS(.*?)</REUTERS>')
id_tokenizer = RegexpTokenizer('NEWID="(.*?)"')
metadata_tokenizer = RegexpTokenizer('<.*?>')
html_entities_tokenizer = RegexpTokenizer('^&.*?;')
postings_list = {}
f = []
reuters_corpus = []


def remove_metadata(text):
    tags = metadata_tokenizer.tokenize(text)
    html_entities = html_entities_tokenizer.tokenize(text)
    metadata = tags + html_entities
    if len(metadata) > 0:
        for element in metadata:
            text = text.replace(element, '')
    return text


while True:
    document = input("Please enter document: ")
    if document == "exit()":  # Exit the loop
        break
    elif document == "reuters_corpus":  # Get all sgm files from the reuters corpus and use that as index (Automatic index creation) (For testing queries)
        for i in range(22):
            doc = f"./reuters21578/reut2-0{i:02d}.sgm"
            try:
                if i != 17:
                    with open(doc, 'rt') as file:
                        file = file.read()
                else:
                    # Needed as file 17 gave me UnicodeDecodeError
                    file = open(doc, mode="rb")
                    file = file.read()
                    file = str(file)
                articles = article_tokenizer.tokenize(file)
                reuters_corpus += articles

                for article in articles:
                    ID = id_tokenizer.tokenize(article)[0]
                    contents = content_tokenizer.tokenize(article)
                    contents = ' '.join(contents)
                    contents = remove_metadata(contents)
                    tokens = word_tokenize(contents)
                    for token in tokens:
                        f.append((ID, token))
            except IOError:
                print("Error: File does not exist")

    else:  # Let user select the file to be added to index
        document = './reuters21578/' + document
        contents = ''
        try:
            with open(document, 'rt') as file:
                contents = file.read()
        except IOError:
            print("Error: File does not exist")
        finally:
            file.close()

        articles = article_tokenizer.tokenize(contents)
        for article in articles:
            ID = id_tokenizer.tokenize(article)[0]
            contents = content_tokenizer.tokenize(article)
            contents = ' '.join(contents)
            contents = remove_metadata(contents)
            # Tokenize article content, need to get title and body
            tokens = word_tokenize(contents)
            for token in tokens:
                f.append((ID, token))

## Sort F and remove duplicates

In [302]:
f.sort(key=lambda x: int(x[0]))
print(f[:300])

[('1', 'BAHIA'), ('1', 'COCOA'), ('1', 'REVIEW'), ('1', 'SALVADOR'), ('1', ','), ('1', 'Feb'), ('1', '26'), ('1', '-'), ('1', 'Showers'), ('1', 'continued'), ('1', 'throughout'), ('1', 'the'), ('1', 'week'), ('1', 'in'), ('1', 'the'), ('1', 'Bahia'), ('1', 'cocoa'), ('1', 'zone'), ('1', ','), ('1', 'alleviating'), ('1', 'the'), ('1', 'drought'), ('1', 'since'), ('1', 'early'), ('1', 'January'), ('1', 'and'), ('1', 'improving'), ('1', 'prospects'), ('1', 'for'), ('1', 'the'), ('1', 'coming'), ('1', 'temporao'), ('1', ','), ('1', 'although'), ('1', 'normal'), ('1', 'humidity'), ('1', 'levels'), ('1', 'have'), ('1', 'not'), ('1', 'been'), ('1', 'restored'), ('1', ','), ('1', 'Comissaria'), ('1', 'Smith'), ('1', 'said'), ('1', 'in'), ('1', 'its'), ('1', 'weekly'), ('1', 'review'), ('1', '.'), ('1', 'The'), ('1', 'dry'), ('1', 'period'), ('1', 'means'), ('1', 'the'), ('1', 'temporao'), ('1', 'will'), ('1', 'be'), ('1', 'late'), ('1', 'this'), ('1', 'year'), ('1', '.'), ('1', 'Arrivals'), ('

In [303]:
f = list(dict.fromkeys(f))

In [304]:
postings_list = {}
for pair in f:
    # Pair[0] is docID and pair[1] is word
    if pair[1] in postings_list:
        postings_list[pair[1]].append(pair[0])
    else:
        postings_list[pair[1]] = [pair[0]]

## Query processor

In [305]:
def query_processor(postings, query):
    res = ''
    if query in postings:
        print(f"Query for word {query} retrieved successfully. The results are:\n{postings[query]}")
        res = postings[query]
    elif query == "exit()":
        print("Thank you for using the query processor. See you next time!")
    else:
        print("Query not found in postings list")
    return res


query = None
while query != "exit()":
    query = input("Please enter the word to search: (Type exit() to stop)")
    result = query_processor(postings_list, query)

Thank you for using the query processor. See you next time!


## Validate 3 queries

In [306]:
query1 = "exit"
query2 = "business"
query3 = "stocks"

test_queries = {query1: query_processor(postings_list, query1),
                query2: query_processor(postings_list, query2),
                query3: query_processor(postings_list, query3)}

# Validate queries
for query in test_queries:
    ids = test_queries[query]
    for number in ids:
        newID = int(number) - 1
        if query in reuters_corpus[newID]:
            print(f"✓ {query} was FOUND in the article with id: {newID + 1}")
        else:
            print(f"✕{query} was NOT FOUND in the article with id: {newID + 1}")


Query for word exit retrieved successfully. The results are:
['386', '583', '643', '2375', '2878', '3196', '3534', '7405', '8325', '8940', '8980', '11471', '16082', '16864', '16904', '20705']
Query for word business retrieved successfully. The results are:
['15', '27', '54', '62', '80', '97', '110', '155', '238', '252', '279', '302', '317', '320', '331', '334', '346', '358', '367', '372', '432', '490', '504', '524', '526', '540', '545', '553', '591', '635', '657', '666', '667', '671', '685', '712', '720', '727', '730', '733', '741', '743', '748', '750', '806', '823', '829', '840', '891', '898', '902', '910', '914', '926', '936', '942', '969', '974', '983', '1002', '1019', '1021', '1027', '1032', '1037', '1041', '1045', '1053', '1080', '1084', '1099', '1159', '1161', '1173', '1183', '1209', '1210', '1252', '1254', '1276', '1278', '1292', '1308', '1353', '1354', '1357', '1374', '1376', '1414', '1421', '1429', '1491', '1498', '1499', '1501', '1502', '1506', '1540', '1581', '1630', '1666',

In [307]:
challenge_queries = ['copper', 'Samjens', 'Carmark', 'Bundesbank']
res_txt = open("challenge_queries_results.txt", "w")
for word in challenge_queries:
    result = query_processor(postings_list, word)
    res_txt.write(f"\nResults for query {word} are the following:\n{result}")

res_txt.close()

Query for word copper retrieved successfully. The results are:
['22', '793', '800', '816', '1148', '1184', '1552', '1607', '2006', '2074', '2186', '2764', '2782', '2880', '3454', '3613', '3862', '4291', '4431', '4744', '4816', '5203', '5209', '5435', '5788', '5827', '5888', '6025', '6225', '6395', '6846', '6927', '7552', '7724', '7775', '8569', '8601', '11273', '11945', '12024', '12215', '12223', '12243', '12489', '12513', '12633', '12641', '12651', '12683', '12692', '12725', '12857', '12861', '12910', '12926', '12963', '12980', '12992', '13003', '13670', '13675', '13678', '13694', '13877', '13897', '14274', '14286', '14297', '14302', '14398', '14476', '14499', '14572', '14687', '14805', '14852', '15166', '15264', '15556', '15831', '15932', '15988', '16123', '16971', '17007', '17118', '18225', '18280', '18317', '18339', '18392', '18394', '18430', '18457', '18631', '18691', '18758', '18849', '18952', '19502', '19781', '19786', '19878', '21293', '21327']
Query for word Samjens retrieved 

## Implement lossy dictionary compression

In [308]:
def find_stop_words(postings):
    sorted_postings = sorted([(len(v), k) for k, v in postings.items()], reverse=True)
    sorted_postings = sorted_postings[:150]
    common_stop_words = [x[1] for x in sorted_postings]
    return common_stop_words

In [309]:
# Distinct terms
unfiltered_terms = [key for key in postings_list]
no_numbers_terms = [word for word in unfiltered_terms if not word.isnumeric()]
case_folding_terms = {w.lower() for w in no_numbers_terms}  # set
stop_words_150 = find_stop_words(postings_list)
stop_words_30 = stop_words_150[:30]
remove_30_stop_words = {w for w in case_folding_terms if w not in stop_words_30}
remove_150_stop_words = {w for w in case_folding_terms if w not in stop_words_150}
porter = PorterStemmer()
porter_stemmed = {porter.stem(word) for word in remove_150_stop_words}

In [310]:
def percentage_diff(num1, num2):
    average = (num1 + num2) / 2
    diff = abs(num1 - num2)
    return int(diff / average * 100)


table_terms = {'Sections': ['unfiltered', 'no numbers', 'case folding', '30 stop words', '150 stop words', 'stemming'],
               'numbers': [len(unfiltered_terms), len(no_numbers_terms), len(case_folding_terms),
                           len(remove_30_stop_words),
                           len(remove_150_stop_words), len(porter_stemmed)],
               '∆%': ['', percentage_diff(len(unfiltered_terms), len(no_numbers_terms)),
                      percentage_diff(len(no_numbers_terms), len(case_folding_terms)),
                      percentage_diff(len(case_folding_terms), len(remove_30_stop_words)),
                      percentage_diff(len(case_folding_terms), len(remove_150_stop_words)),
                      percentage_diff(len(remove_150_stop_words), len(porter_stemmed))],
               'T%': ['', percentage_diff(len(unfiltered_terms), len(no_numbers_terms)),
                      percentage_diff(len(unfiltered_terms), len(case_folding_terms)),
                      percentage_diff(len(unfiltered_terms), len(remove_30_stop_words)),
                      percentage_diff(len(unfiltered_terms), len(remove_150_stop_words)),
                      percentage_diff(len(unfiltered_terms), len(porter_stemmed))]}
print(tabulate(table_terms, headers="keys", tablefmt='fancy_grid', missingval='N/A'))

╒════════════════╤═══════════╤══════╤══════╕
│ Sections       │   numbers │ ∆%   │ T%   │
╞════════════════╪═══════════╪══════╪══════╡
│ unfiltered     │    111232 │      │      │
├────────────────┼───────────┼──────┼──────┤
│ no numbers     │    108527 │ 2    │ 2    │
├────────────────┼───────────┼──────┼──────┤
│ case folding   │     90969 │ 17   │ 20   │
├────────────────┼───────────┼──────┼──────┤
│ 30 stop words  │     90942 │ 0    │ 20   │
├────────────────┼───────────┼──────┼──────┤
│ 150 stop words │     90858 │ 0    │ 20   │
├────────────────┼───────────┼──────┼──────┤
│ stemming       │     79808 │ 12   │ 32   │
╘════════════════╧═══════════╧══════╧══════╛


In [311]:
def len_posts(postings):
    length = 0
    for key in postings:
        length += len(postings[key])
    length += len(postings)
    return length


unfiltered_postings = len_posts(postings_list)

# Remove numbers
for word in unfiltered_terms:
    if word.isnumeric():
        del postings_list[word]

no_numbers_postings = len_posts(postings_list)

In [312]:
# Casefold
for word in no_numbers_terms:
    if word.lower() in postings_list:
        if postings_list[word.lower()] != postings_list[word]:
            # Find union of lists
            union = list(set().union(postings_list[word.lower()], postings_list[word]))
            postings_list[word.lower()] = union
            del postings_list[word]
    else:
        postings_list[word.lower()] = postings_list[word]
        del postings_list[word]

case_folding_postings = len_posts(postings_list)

In [313]:
# Remove 30 stop words
for word in stop_words_30:
    if word in postings_list:
        del postings_list[word]

remove_30_stop_words_postings = len_posts(postings_list)
# Remove 150 stop words
for word in stop_words_150:
    if word in postings_list:
        del postings_list[word]

remove_150_stop_words_postings = len_posts(postings_list)

In [314]:
# Stemming
for word in remove_150_stop_words:
    if porter.stem(word) in postings_list:
        if postings_list[porter.stem(word)] != postings_list[word]:
            # Find union of lists
            union = list(set().union(postings_list[porter.stem(word)], postings_list[word]))
            postings_list[porter.stem(word)] = union
            del postings_list[word]
    else:
        postings_list[porter.stem(word)] = postings_list[word]
        del postings_list[word]
stem_postings = len_posts(postings_list)

In [315]:
table_terms = {'Sections': ['unfiltered', 'no numbers', 'case folding', '30 stop words', '150 stop words', 'stemming'],
               'numbers': [unfiltered_postings, no_numbers_postings, case_folding_postings,
                           remove_30_stop_words_postings,
                           remove_150_stop_words_postings, stem_postings],
               '∆%': ['', percentage_diff(unfiltered_postings, no_numbers_postings),
                      percentage_diff(no_numbers_postings, case_folding_postings),
                      percentage_diff(case_folding_postings, remove_30_stop_words_postings),
                      percentage_diff(remove_30_stop_words_postings, remove_150_stop_words_postings),
                      percentage_diff(remove_150_stop_words_postings, stem_postings)],
               'T%': ['', percentage_diff(unfiltered_postings, no_numbers_postings),
                      percentage_diff(unfiltered_postings, case_folding_postings),
                      percentage_diff(unfiltered_postings, remove_30_stop_words_postings),
                      percentage_diff(unfiltered_postings, remove_150_stop_words_postings),
                      percentage_diff(unfiltered_postings, stem_postings)]}
print(tabulate(table_terms, headers="keys", tablefmt='fancy_grid', missingval='N/A'))

╒════════════════╤═══════════╤══════╤══════╕
│ Sections       │   numbers │ ∆%   │ T%   │
╞════════════════╪═══════════╪══════╪══════╡
│ unfiltered     │   2064926 │      │      │
├────────────────┼───────────┼──────┼──────┤
│ no numbers     │   1985282 │ 3    │ 3    │
├────────────────┼───────────┼──────┼──────┤
│ case folding   │   1849177 │ 7    │ 11   │
├────────────────┼───────────┼──────┼──────┤
│ 30 stop words  │   1534023 │ 18   │ 29   │
├────────────────┼───────────┼──────┼──────┤
│ 150 stop words │   1275849 │ 18   │ 47   │
├────────────────┼───────────┼──────┼──────┤
│ stemming       │   1214821 │ 4    │ 51   │
╘════════════════╧═══════════╧══════╧══════╛


In [316]:
res_txt = open("challenge_queries_results_2.txt", "w")
for word in challenge_queries:
    result = query_processor(postings_list, word)
    res_txt.write(f"\nResults for query {word} are the following:\n{result}")

res_txt.close()

Query for word copper retrieved successfully. The results are:
['16123', '1552', '5827', '18317', '19878', '8601', '13518', '2074', '18952', '13536', '13463', '800', '13681', '11945', '14572', '19781', '13734', '5788', '21327', '3613', '5203', '7775', '14122', '3862', '14036', '5209', '14302', '13840', '14274', '17118', '14431', '12692', '4816', '14127', '2782', '6225', '12243', '4291', '13465', '18280', '13678', '20096', '22', '21293', '2006', '1607', '13495', '13724', '13932', '12215', '6025', '12683', '14687', '11273', '12980', '14452', '8169', '12641', '18457', '13927', '13810', '6395', '14805', '12857', '13605', '17007', '13635', '14136', '4744', '15166', '13980', '14066', '12910', '15264', '19502', '13665', '12223', '19786', '14297', '14064', '14142', '13757', '15932', '13003', '15831', '13687', '2880', '1184', '5888', '793', '18394', '2186', '12513', '13566', '12489', '15988', '14286', '13563', '12725', '18339', '12633', '18631', '14405', '2764', '13897', '13670', '14852', '1202

In [317]:
for query in test_queries:
    ids = test_queries[query]
    for number in ids:
        newID = int(number) - 1
        if query in reuters_corpus[newID]:
            print(f"✓ {query} was FOUND in the article with id: {newID + 1}")
        else:
            print(f"✕{query} was NOT FOUND in the article with id: {newID + 1}")


✓ exit was FOUND in the article with id: 386
✓ exit was FOUND in the article with id: 583
✓ exit was FOUND in the article with id: 643
✓ exit was FOUND in the article with id: 2375
✓ exit was FOUND in the article with id: 2878
✓ exit was FOUND in the article with id: 3196
✓ exit was FOUND in the article with id: 3534
✓ exit was FOUND in the article with id: 7405
✓ exit was FOUND in the article with id: 8325
✓ exit was FOUND in the article with id: 8940
✓ exit was FOUND in the article with id: 8980
✓ exit was FOUND in the article with id: 11471
✓ exit was FOUND in the article with id: 16082
✓ exit was FOUND in the article with id: 16864
✓ exit was FOUND in the article with id: 16904
✓ exit was FOUND in the article with id: 20705
✓ business was FOUND in the article with id: 15
✓ business was FOUND in the article with id: 27
✓ business was FOUND in the article with id: 54
✓ business was FOUND in the article with id: 62
✓ business was FOUND in the article with id: 80
✓ business was FOUND i